# Analyse du dossier outputs

Ce notebook vérifie que le dossier `outputs` contient bien :
- 3000 scénarios JSON
- 6000 CSV dans `dispatch_batch`
- 6000 plots dans `plots`
- 3000 reports JSON dans `reports`

In [ ]:
# Monte Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# Définir le chemin root vers le repo benchmark sur Google Drive
ROOT = '/content/drive/MyDrive/benchmark'
OUTPUTS_DIR = os.path.join(ROOT, 'outputs')

print(f"Root: {ROOT}")
print(f"Outputs: {OUTPUTS_DIR}")
print(f"Outputs existe: {os.path.exists(OUTPUTS_DIR)}")

In [ ]:
from pathlib import Path

def count_files(directory, extension=None):
    """Compte les fichiers dans un dossier avec une extension optionnelle."""
    path = Path(directory)
    if not path.exists():
        return 0, []
    
    if extension:
        files = list(path.glob(f'*.{extension}'))
    else:
        files = [f for f in path.iterdir() if f.is_file()]
    
    return len(files), files

def check_status(actual, expected, name):
    """Affiche le statut de vérification."""
    status = "✅" if actual == expected else "❌"
    return f"{status} {name}: {actual}/{expected}"

In [ ]:
# Vérification des scénarios JSON (racine de outputs)
scenarios_count, scenarios = count_files(OUTPUTS_DIR, 'json')
print(check_status(scenarios_count, 3000, "Scénarios JSON (outputs/)"))

In [ ]:
# Vérification des CSV dans dispatch_batch
dispatch_dir = os.path.join(OUTPUTS_DIR, 'dispatch_batch')
csv_count, csv_files = count_files(dispatch_dir, 'csv')
print(check_status(csv_count, 6000, "CSV (dispatch_batch/)"))

In [ ]:
# Vérification des plots
plots_dir = os.path.join(OUTPUTS_DIR, 'plots')
# Compter tous les fichiers image (png, jpg, pdf, svg)
plots_path = Path(plots_dir)
if plots_path.exists():
    plots = list(plots_path.glob('*.*'))
    plots_count = len([f for f in plots if f.is_file()])
else:
    plots_count = 0
print(check_status(plots_count, 6000, "Plots (plots/)"))

In [ ]:
# Vérification des reports JSON
reports_dir = os.path.join(OUTPUTS_DIR, 'reports')
reports_count, reports = count_files(reports_dir, 'json')
print(check_status(reports_count, 3000, "Reports JSON (reports/)"))

# Identification des fichiers manquants\n\nLes cellules suivantes identifient les fichiers manquants en se basant sur l'ordre numérique (1 à 3000).

In [ ]:
import re

def extract_numbers_from_files(files):
    """Extrait les numéros des noms de fichiers."""
    numbers = set()
    for f in files:
        # Cherche tous les nombres dans le nom du fichier
        matches = re.findall(r'\d+', f.stem if hasattr(f, 'stem') else str(f))
        if matches:
            # Prend le premier nombre trouvé
            numbers.add(int(matches[0]))
    return numbers

def find_missing_numbers(found_numbers, expected_range):
    """Trouve les numéros manquants dans une plage attendue."""
    expected = set(range(expected_range[0], expected_range[1] + 1))
    missing = sorted(expected - found_numbers)
    return missing

def display_missing(missing, name, max_display=50):
    """Affiche les numéros manquants."""
    if not missing:
        print(f"✅ {name}: Aucun fichier manquant !")
    else:
        print(f"❌ {name}: {len(missing)} fichiers manquants")
        if len(missing) <= max_display:
            print(f"   Numéros manquants: {missing}")
        else:
            print(f"   Premiers {max_display}: {missing[:max_display]}")
            print(f"   ... et {len(missing) - max_display} autres")

In [ ]:
# Scénarios JSON manquants (1-3000)
scenarios_numbers = extract_numbers_from_files(scenarios)
missing_scenarios = find_missing_numbers(scenarios_numbers, (1, 3000))
display_missing(missing_scenarios, "Scénarios JSON")

In [ ]:
# CSV dispatch_batch manquants (2 fichiers par scénario: 1-3000 → 6000 fichiers)
# On s'attend à des fichiers pour chaque scénario (1-3000)
csv_numbers = extract_numbers_from_files(csv_files)
missing_csv = find_missing_numbers(csv_numbers, (1, 3000))
display_missing(missing_csv, "CSV dispatch_batch (numéros de scénarios)")

In [ ]:
# Plots manquants (2 plots par scénario: 1-3000 → 6000 fichiers)
plots_numbers = extract_numbers_from_files(plots if 'plots' in dir() else [])
missing_plots = find_missing_numbers(plots_numbers, (1, 3000))
display_missing(missing_plots, "Plots (numéros de scénarios)")

In [ ]:
# Reports JSON manquants (1-3000)
reports_numbers = extract_numbers_from_files(reports)
missing_reports = find_missing_numbers(reports_numbers, (1, 3000))
display_missing(missing_reports, "Reports JSON")

In [ ]:
# Résumé des fichiers manquants
print("="*50)
print("RÉSUMÉ DES FICHIERS MANQUANTS")
print("="*50)

all_missing = set(missing_scenarios) | set(missing_csv) | set(missing_plots) | set(missing_reports)
if all_missing:
    print(f"\nNuméros de scénarios avec au moins un fichier manquant: {len(all_missing)}")
    print(f"Liste: {sorted(all_missing)[:100]}{'...' if len(all_missing) > 100 else ''}")
else:
    print("\n🎉 Tous les fichiers sont présents pour tous les scénarios !")

In [ ]:
# Résumé final
print("\n" + "="*50)
print("RÉSUMÉ DE L'ANALYSE")
print("="*50)
print(check_status(scenarios_count, 3000, "Scénarios JSON"))
print(check_status(csv_count, 6000, "CSV dispatch_batch"))
print(check_status(plots_count, 6000, "Plots"))
print(check_status(reports_count, 3000, "Reports JSON"))
print("="*50)

total_expected = 3000 + 6000 + 6000 + 3000
total_actual = scenarios_count + csv_count + plots_count + reports_count
print(f"\nTotal: {total_actual}/{total_expected} fichiers")

if total_actual == total_expected:
    print("\n🎉 Tous les fichiers sont présents !")
else:
    print(f"\n⚠️ Il manque {total_expected - total_actual} fichiers.")